In [1]:
import os
import numpy as np
import pandas as pd
from scipy.stats import poisson  # 泊松分布

In [2]:
# 英超使用(2018-2019)資料評估(2019-2020)賽季比分分布狀況
# TODO 資料整理
teams = pd.read_csv(os.getcwd() + '/data/82/teams.csv')
teams.set_index(["team_id"], inplace=True)

matches = pd.read_csv(os.getcwd() + '/data/82/matches.csv')
lastSeason = matches[matches['season_id'] == 388]
curSeason = matches[matches['season_id'] == 5073]

# 主客隊最大進球數
homeMax = lastSeason['home_score'].max()
awayMax = lastSeason['home_score'].max()

# curSeason.set_index('match_id')
curSeason = curSeason.drop(columns=['home_half_score', 'home_red', 'home_yellow', 'home_corner','away_half_score', 'away_red', 'away_yellow', 'away_corner', 'season_id', 'season_name'])

In [3]:
# TODO 計算平均進球數
# 聯盟主隊平均主場進球數 = 主場賽季總進球數/比賽場數（賽季）
seasonHomeAvg = lastSeason['home_score'].sum() / lastSeason.shape[0]
# 聯盟主隊平均客場進球數 = 客場賽季總進球數/比賽場數（賽季）
seasonAwayAvg = lastSeason['away_score'].sum() / lastSeason.shape[0]

print('主場平均進球數: %s 球' % seasonHomeAvg)
print('客場平均進球數: %s 球' % seasonAwayAvg)

# TODO 統計資料
# 隊伍為主場平均進球
temp = lastSeason.groupby(['home_team'])['home_score', 'away_score'].sum()
temp.index.rename('team_id')
teams = teams.join(temp, on='team_id')
teams = teams.rename(columns={"home_score": "homeGetScore", "away_score": "homeLostScore"})

# 隊伍為主場總場次
temp = lastSeason.groupby(['home_team']).size()
temp.index.rename('team_id')
temp = temp.rename('homeMatchCount')
teams = teams.join(temp, on='team_id')

# 隊伍為客場平均進球
temp = lastSeason.groupby(['away_team'])['home_score', 'away_score'].sum()
temp.index.rename('team_id')
teams = teams.join(temp, on='team_id')
teams = teams.rename(columns={"home_score": "awayLostScore", "away_score": "awayGetScore"})

# 隊伍為客場總場次
temp = lastSeason.groupby(['away_team']).size()
temp.index.rename('team_id')
temp = temp.rename('awayMatchCount')
teams = teams.join(temp, on='team_id')

# 删除有NaN rows
teams = teams.dropna() 

主場平均進球數: 1.4921052631578948 球
客場平均進球數: 1.2078947368421054 球


In [4]:
# TODO 計算進攻率(上賽季)
# 主隊主場平均進球數 = 主隊主場進球數 / 主場賽事場數。
# 主隊主場平均進球數 / 聯盟賽季主場平均進球數
teams['homeAttack'] =  teams['homeGetScore'] / teams['homeMatchCount'] / seasonHomeAvg
teams['awayAttack'] =  teams['awayGetScore'] / teams['awayMatchCount'] / seasonAwayAvg

# TODO 計算防守率(上賽季)
# 客隊客場平均失球數 = 客隊客場失球數 / 客場賽事場數。
# 客隊客場平均失球數 / 聯盟賽季客場平均失球數
teams['homeDefense'] =  teams['homeLostScore'] / teams['homeMatchCount'] / seasonAwayAvg
teams['awayDefense'] =  teams['awayLostScore'] / teams['awayMatchCount'] / seasonHomeAvg

In [5]:
teams

,Unnamed: 0,competition_id,short_name_zh,short_name_en,market_value,homeGetScore,homeLostScore,homeMatchCount,awayLostScore,awayGetScore,awayMatchCount,homeAttack,awayAttack,homeDefense,awayDefense
team_id,,,,,,,,,,,,,,,
10022,3,82,沃特福德,Watford,235150000,20.0,19.0,19.0,31.0,20.0,19.0,0.705467,0.871460,0.827887,1.093474
10115,4,82,伯恩茅斯,Bournemouth AFC,334250000,23.0,34.0,19.0,33.0,22.0,19.0,0.811287,0.958606,1.481481,1.164021
10135,5,82,曼联,Manchester United,753250000,27.0,9.0,19.0,26.0,22.0,19.0,0.952381,0.958606,0.392157,0.917108
10136,6,82,纽卡斯尔联,Newcastle United,273580000,32.0,24.0,19.0,41.0,12.0,19.0,1.128748,0.522876,1.045752,1.446208
10176,7,82,西汉姆联,West Ham United,348750000,34.0,26.0,19.0,25.0,31.0,19.0,1.199295,1.350763,1.132898,0.881834
10180,8,82,切尔西,Chelsea,807000000,32.0,30.0,19.0,23.0,27.0,19.0,1.128748,1.176471,1.307190,0.811287
10181,9,82,曼城,Manchester City,1280000000,47.0,21.0,19.0,20.0,24.0,19.0,1.657848,1.045752,0.915033,0.705467
10182,10,82,南安普顿,Southampton,237000000,39.0,22.0,19.0,19.0,20.0,19.0,1.375661,0.871460,0.958606,0.670194
10183,11,82,热刺,Tottenham Hotspur,985000000,35.0,15.0,19.0,20.0,34.0,19.0,1.234568,1.481481,0.653595,0.705467


In [6]:
# 整理當前賽季資料 準備進行預測
integration = pd.merge(curSeason, teams.loc[:,['homeAttack','homeDefense']], left_on='home_team', right_on='team_id')

integration = pd.merge(integration, teams.loc[:,['awayAttack','awayDefense']], left_on='away_team', right_on='team_id')

# # TODO 計算可能進球數
# 主隊可能進球數 = 主隊進攻率 * 客隊防守率 * 聯盟主隊平均主場進球數
integration['lambdaHome'] = integration['homeAttack'] * integration['awayDefense'] * seasonHomeAvg
# # 客隊可能進球數 = 客隊進攻率 * 主隊防守率 * 聯盟客隊平均客場進球數
integration['lambdaAway'] = integration['awayAttack'] * integration['homeDefense'] * seasonAwayAvg

# # curSeason['awayPoisson'] = poisson.pmf(np.arange(awayMax + 1), lambdaAway)

# integration.loc[integration.loc[:, "match_id"] == 2004333]
integration

,Unnamed: 0,match_id,competition_id,open_date,home_team,away_team,home_rank,home_score,away_rank,away_score,homeAttack,homeDefense,awayAttack,awayDefense,lambdaHome,lambdaAway
0,4970,2004087,82,1471186800,10215,10249,2,3,8,4,1.093474,0.479303,1.307190,0.987654,1.611436,0.756794
1,5149,2004267,82,1480858200,10115,10249,12,4,3,3,0.811287,1.481481,1.307190,0.987654,1.195582,2.339181
2,5009,2004127,82,1474052400,10180,10249,2,1,6,2,1.128748,1.307190,1.307190,0.987654,1.663418,2.063983
3,5109,2004227,82,1477758600,10244,10249,11,2,3,4,0.670194,1.002179,1.307190,0.987654,0.987654,1.582387
4,5192,2004310,82,1489941000,10181,10249,3,1,4,1,1.657848,0.915033,1.307190,0.987654,2.443145,1.444788
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
151,5158,2004276,82,1480786200,10176,10215,16,1,4,5,1.199295,1.132898,1.481481,0.881834,1.578019,2.027290
152,5135,2004253,82,1481658300,10250,10215,9,2,2,1,1.234568,1.307190,1.481481,0.881834,1.624431,2.339181
153,5306,2004424,82,1493566200,10183,10215,2,2,6,0,1.234568,0.653595,1.481481,0.881834,1.624431,1.169591
154,5091,2004209,82,1479558600,10135,10215,6,1,4,1,0.952381,0.392157,1.481481,0.881834,1.253133,0.701754


In [7]:
# TODO 使用Poisson計算比分分布
scorePred =  {
        '0:0': {},
        '0:1': {},
        '0:2': {},
        '0:3': {},
        '1:0': {},
        '1:1': {},
        '1:2': {},
        '1:3': {},
        '2:0': {},
        '2:1': {},
        '2:2': {},
        '2:3': {},
        '3:0': {},
        '3:1': {},
        '3:2': {},
        '3:3': {},
        '主隊三球以上': {},
        '客隊三球以上':{},
        '其他': {},
    }

for index, row in integration.iterrows():
    homePoisson = poisson.pmf(np.arange(homeMax + 1), row['lambdaHome'])
    awayPoisson = poisson.pmf(np.arange(awayMax + 1), row['lambdaAway'])

    matchId = row['match_id']
    
    for i in np.arange(homeMax + 1):
        for j in np.arange(awayMax + 1):

            score = str(i) + ':' + str(j)
            
            if i > 3 and j > 3:
                score = '其他'
            elif i > 3:
                score = '主隊三球以上'
            elif j > 3:
                score = '客隊三球以上'

            prediction = homePoisson[i] * awayPoisson[j] * 100

            if not score == str(i) + ':' + str(j) and matchId in scorePred[score]:
                prediction = scorePred[score][matchId] + prediction

            scorePred[score][matchId] = prediction

    # result[] = scorePred

In [8]:
test = pd.DataFrame.from_dict(scorePred)
test

,0:0,0:1,0:2,0:3,1:0,1:1,1:2,1:3,2:0,2:1,2:2,2:3,3:0,3:1,3:2,3:3,主隊三球以上,客隊三球以上,其他
2004087,9.364634,7.087098,2.681737,0.676507,15.090508,11.420405,4.321447,1.090148,12.158694,9.201626,3.481867,0.878352,6.530986,4.942610,1.870269,0.471803,7.841300,0.690209,0.059299
2004267,2.916567,6.822380,7.979391,6.221748,3.486994,8.156711,9.540013,7.438607,2.084493,4.876007,5.702932,4.446731,0.830727,1.943221,2.272773,1.772143,2.622588,19.184368,0.657756
2004127,2.405527,4.964967,5.123805,3.525150,4.001396,8.258815,8.523029,5.863797,3.327997,6.868930,7.088679,4.876972,1.845283,3.808634,3.930478,2.704148,7.281668,13.614561,1.285526
2004227,7.653236,12.110383,9.581659,5.053965,7.558751,11.960872,9.463367,4.991570,3.732717,5.906604,4.673267,2.464973,1.228878,1.944561,1.538524,0.811514,1.677563,7.377907,0.136498
2004310,2.048764,2.960031,2.138309,1.029802,5.005428,7.231785,5.224199,2.515954,6.114493,8.834149,6.381738,3.073421,4.979531,7.194369,5.197170,2.502937,20.485318,4.476222,1.265970
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2004276,2.717903,5.509979,5.585164,3.774250,4.288903,8.694852,8.813495,5.955838,3.383985,6.860321,6.953932,4.699213,1.779998,3.608573,3.657812,2.471816,6.355256,13.212466,1.066186
2004253,1.899437,4.443127,5.196640,4.051961,3.085505,7.217555,8.441585,6.582133,2.506096,5.862212,6.856388,5.346112,1.356994,3.174254,3.712578,2.894797,6.390982,18.215245,1.602885
2004424,6.117467,7.154932,4.184171,1.631256,9.937406,11.622696,6.796899,2.649863,8.071317,9.440137,5.520548,2.152260,4.370434,5.111618,2.989251,1.165400,7.824299,2.842912,0.250167
2004209,14.158044,9.935470,3.486130,0.815469,17.741910,12.450463,4.368584,1.021891,11.116485,7.801042,2.737208,0.640283,4.643477,3.258581,1.143362,0.267453,3.801506,0.557094,0.022156


In [10]:
# TODO 使用歷史資料進行驗證

In [9]:
# TODO 結果呈現
# 所有比分發生機率
# 依實際分布情況排序
# 依預估分布情況排序